In [1]:
from __future__ import print_function
import numpy as np
from nltk.stem import SnowballStemmer
from gensim.models import Word2Vec
from gensim import corpora
from collections import defaultdict
import sys
import pickle
import os

UNDEFINED_TOKEN = "undefined_token"
MAX_WORD_LENGTH = 20
UNDEFINED = "_"
stemmer = SnowballStemmer("russian")

In [25]:
class LowerSentencesWithoutStops(object):
    def __init__(self, fnames, token2token, stops):
        self.fnames = fnames        
        self.token2token = token2token         
        
    def __iter__(self):
        for fname in self.fnames:
            for line in open(fname, 'r', encoding="utf8"):            
                yield [self.token2token.get(token, UNDEFINED_TOKEN)
                       for token in line.lower().split()]
                
                

    
def split_word(word, stemmer):
    flex = word[len(stemmer.stem(word)):]
    if len(flex):
        return word[:-len(flex)], flex
    return word, "empty"


def build_vocab(sentences, min_freq=0, max_size=10000, undefined_id=0):
    """ 
    Строит словарь из слов встертившихся более min_freq раз,
    но размеров  не более max_size, в случае бОльшего количества токенов
    отбрасываются менее частотные токены, undefined_id - id первого токена в словаре,
    который будет называться "undefined_token"
    """
    offset = undefined_id
    token2id = {UNDEFINED_TOKEN: offset}
    id2token = {offset: UNDEFINED_TOKEN}    
    
    counter = defaultdict(int)    
    for sentence in sentences:
        for token in sentence:
            counter[token] += 1
    sorted_tokens = [t_f[0]  for t_f in 
                     sorted([t_f for t_f in counter.items() if t_f[1] >= min_freq],
                           key=lambda tf: -tf[1])]                     
    
    for token in sorted_tokens[:max_size - len(token2id)]:
        offset += 1
        token2id[token] = offset
        id2token[offset] = token
    return token2id, id2token 


def build_ch_vocab(text, min_freq=0, max_size=100, undefined_id=0):
    """ 
    Строит словарь из слов встертившихся более min_freq раз,
    но размеров  не более max_size, в случае бОльшего количества токенов
    отбрасываются менее частотные токены, undefined_id - id первого токена в словаре,
    который будет называться "undefined_token"
    """
    offset = undefined_id
    token2id = {UNDEFINED: offset}
    id2token = {offset: UNDEFINED}    
    
    counter = defaultdict(int)    
    for token in text:
        counter[token] += 1
        
    sorted_tokens = [t_f[0]  for t_f in 
                     sorted([t_f for t_f in counter.items() if t_f[1] >= min_freq],
                           key=lambda tf: -tf[1]) if t_f[0] not in token2id]                     
    
    for token in sorted_tokens[:max_size - len(token2id)]:
        offset += 1
        token2id[token] = offset
        id2token[offset] = token
    return token2id, id2token  




def read_gikrya(path):
    """
    Reading format:
    row_index<TAB>form<TAB>lemma<TAB>POS<TAB>tag
    """
    
    morpho_map = {"POS":{UNDEFINED: 0, 
                         0: UNDEFINED}}
    sentences = []
    vocab = {}    
    with open(path, 'r') as f:
        
        sentence = []
        for line in f:
            splits = line.strip().split('\t')      
            if len(splits) == 4:
                splits.insert(0, 1)
            if len(splits) == 5:
                form, lemma, POS, tags = splits[1:]
                if POS not in  morpho_map["POS"]:
                    morpho_map["POS"][POS] = len(morpho_map["POS"]) // 2 
                    morpho_map["POS"][morpho_map["POS"][POS]] =  POS
                tags_list = [("POS", POS)]
                if tags != "_":
                    for tag_val in tags.split("|"):
                        tag, val = tag_val.split("=")
                        tags_list.append((tag, val))
                        if tag not in morpho_map:
                            morpho_map[tag] = {UNDEFINED: 0,
                                               0: UNDEFINED}
                        if val not in morpho_map[tag]:
                            morpho_map[tag][val] = len(morpho_map[tag]) // 2 
                            morpho_map[tag][morpho_map[tag][val]] = val
                if form not in vocab:
                    vocab[form] = form
                sentence.append((vocab[form], lemma, tags_list) )
            elif len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
    return sentences, morpho_map 


def read_corpus(path):
    sentences = []
    with open(path, 'r') as f:
        for line in f:
            sentences.append(line.strip().lower().split())
    return sentences
        
    
def write_vecs(path, vecs_path, id2token, w2v_model):

    # косяк с тем чтон undefined token не 0
    vecs = np.zeros(shape=(len(token2id), w2v_model.vector_size))
    with open(path, 'w') as f:
        for tid in range(len(id2token)):
            vecs[tid, :] = w2v_model[id2token[tid]]
            f.write(id2token[tid])
            f.write("\n")
    np.save(vecs_path, vecs)
    

def preproc_dataset(full_tag_sentences, stemmer):    
    sentences = []
    flexes = []
    token_tags = []
    
    for sent in full_tag_sentences:
        temp_sent = []
        temp_flexes = []
        for token_info in sent:
            token = token_info[0].lower()          
            splits = split_word(token, stemmer)
            temp_sent.append(splits[0])
            temp_flexes.append(splits[1])
            token_tags.append(token_info[2])  # надо бы переделать под стиль sentences или?          
        sentences.append(temp_sent)
        flexes.append(temp_flexes)    
    return sentences, flexes, token_tags


def get_tokens(sentences):
    tokens = []
    for sent in sentences:
        for token in sent:
            tokens.append(token)
    return tokens
    
    
def preproc_files(fnames):
    sentences_full = []
    for fname in fnames:
        s_full, _ = read_gikrya(fname)
        sentences_full = sentences_full + s_full
    return sentences_full
            
    

In [3]:
fnames = ["../JointMorphoClosed.txt", 
          "../morphoRuEval-2017/test_collection/VK.txt",
         "../morphoRuEval-2017/test_collection/JZ.txt",
          "../morphoRuEval-2017/test_collection/Lenta.txt"]

sentences_full = preproc_files(fnames)

In [127]:
# morpho_map
# !head  "../JointMorphoClosed.txt"
# len(stem_modeiil.vocab)
class MyStem(object):
    def stem(self, token):
        return token

    
st = MyStem()    
sentences, flexes, token_tags = preproc_dataset(sentences_full, st)

In [128]:
sentences = [[token.replace('\ufeff','') for token in sent] for sent in sentences]
sentences[0][0]

'кстати'

In [33]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Embedding
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
from collections import Counter
# http://192.168.14.116:8888/notebooks/projects/Morph/notebooks/CharLevelTextGenerator.ipynb#

In [129]:
def change_ch(ch, ch2id):
    if ch in ch2id:
        return ch
    return UNDEFINED


text = ' '.join([' '.join(sent) for sent in sentences])

ch2id, id2ch = build_ch_vocab(text, max_size=50)
sents = sentences

text = ''.join([change_ch(ch, ch2id) for ch in text])
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 50


In [198]:
# cut the text in semi-redundant sequences of maxlen characters
chars = sorted(list(set(text)))
chars.append(UNDEFINED)
maxlen = 32
step = 250
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen), dtype=np.int)
y = np.zeros((len(sentences), len(ch2id)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t] = ch2id[char]
        y[i, ch2id[next_chars[i]]] = 1.

nb sequences: 110281
Vectorization...


In [214]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(Embedding(input_dim=len(ch2id), output_dim=32))
model.add(LSTM(256, input_shape=(maxlen, len(ch2id))))
# model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dropout(p=0.25))
model.add(Dense(len(ch2id)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.005)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Build model...


In [250]:
model.save('../models/lstm_model1.h5')

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=512,
              nb_epoch=1, verbose=2)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 1.2]:
        print()
        print('----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        for i in range(400):
            x = np.zeros((1, maxlen))
            for t, char in enumerate(sentence):
                x[0, t] =  ch2id[char]
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = id2ch[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
27s - loss: 1.5254

----- diversity: 0.2
----- Generating with seed: "ых ракетных установок . имена не"
ых ракетных установок . имена не сводом и картор , которых был подале не просто в ней , потому что сполове подорожде не продолжение . под концерта по этому крупные страны . в составал . все верить свое закон и верес случая стактера было образоманность по просто , но ведь в поднему высоком долго , что понять от этому поленно всего подумал , что не пример , в подеренном сторону , и с просте , которые , как у него подале не подобно

----- diversity: 1.2
----- Generating with seed: "ых ракетных установок . имена не"
ых ракетных установок . имена незазачабывать , что мое же языча растер пясы по в_бот начинаются тлюдек отюдыв толкая м нрай заходили _ _ вокяби оправкан алицнуга к юснове , хибудер авелию . помощё они протеной грими к эксперти п эвслост к в переходя , но не хотиствующие тюх в принихали меди черед не 1_ 

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:18: RuntimeWarning: divide by zero encountered in log


ения миники ,

--------------------------------------------------
Iteration 45
Epoch 1/1
26s - loss: 1.6411

----- diversity: 0.2
----- Generating with seed: "главных персонажей разворачивающ"
главных персонажей разворачивающих сонен . он не под весто не принять , что в просто станивают по простот . и в ней вот не на выстаковала , в том , что со сторонно , в тему принимания , по всему принимание . в полного под котором против просто в просто простотной проводительно принимания в настоящем принимает под изрения , на сколы не полного страшно прозойвах продолжить в полного подобрустели . в просто стало под котором не пол

----- diversity: 1.2
----- Generating with seed: "главных персонажей разворачивающ"
главных персонажей разворачивающегому спечаю с кищах : межят в дысаны я  -  пешен на онужествениности . оно все меоем занимая по этенному укв"тить это поджетивают однеждыриеть в рран идёлакии разосказывать рукии она ресуссека . только уживает уцедную читать автолагнский тукокраж за твою в

In [225]:
with open("char2id", "w") as f:
    for i in range(len(ch2id)):
        f.write("{}\n".format(id2ch[i]))


In [259]:
model.save_weights('../models/lstm_model1_weights.h5',)

In [263]:
pickle.dump(model.layers[1].get_weights(), 
            open('../models/lstm_model1_lstm_weights.pkl', 'wb'))

In [269]:
pickle.dump(model.layers[0].get_weights(), 
            open('../models/lstm_model1_lstm_embedd.pkl', 'wb'))
# model.layers[1].get_weights()[1][10:15, 15]

In [265]:
model.layers[0].get_weights()


[array([[  8.08655500e-01,   9.58728254e-01,  -3.17508638e-01, ...,
          -4.73814815e-01,  -1.20089546e-01,   1.94145179e+00],
        [  4.32156950e-01,   9.14899111e-01,   1.34403419e+00, ...,
          -6.99849683e-04,   3.68139863e-01,   1.32854581e-01],
        [  4.83723402e-01,  -4.80563670e-01,   3.52710225e-02, ...,
           8.28634143e-01,  -2.64349014e-01,   3.71212959e-01],
        ..., 
        [  5.44837892e-01,   3.80831957e-02,   9.62517083e-01, ...,
           4.19646680e-01,  -8.39632213e-01,   7.21821368e-01],
        [  8.74767154e-02,   4.27852392e-01,  -6.44073123e-03, ...,
           3.12367082e-01,   2.01106682e-01,   1.24559379e+00],
        [  1.13313961e+00,  -2.08620518e-01,  -1.92607731e-01, ...,
           6.82426453e-01,  -1.03674722e+00,   1.85779297e+00]], dtype=float32)]